In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Extraire les textes issus des documents pdf
---

In [ ]:
# Bouton pour uploader un fichier au début du notebook
from google.colab import files
uploaded = files.upload()

In [ ]:
# Chemin vers le fichier PDF
pdf_path = "/content/Acceuil_affichage_Mairie de Triffouillis sur Loire.pdf"

### markitdown

In [ ]:
!pip install markitdown --quiet

In [ ]:
from markitdown import MarkItDown

md = MarkItDown()
result = md.convert(pdf_path)
print(result.text_content)

### PyPDF2

In [ ]:
!pip install PyPDF2 --quiet

In [ ]:
import PyPDF2

# Ouvrir le fichier PDF en mode lecture binaire
with open(pdf_path, "rb") as pdf_file:
    # Créer un lecteur PDF
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Extraire le texte de chaque page
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()

    # Afficher le texte extrait
    print(text)

### pymupdf4llm

In [ ]:
!pip install pymupdf4llm --quiet

In [ ]:
import pymupdf4llm

md_text = pymupdf4llm.to_markdown(pdf_path)
print("=== Contenu Markdown extrait avec PyMuPDF4LLM ===")
print(md_text)

## Océriser les textes contenus dans les images

---



In [ ]:
# Bouton pour uploader un fichier au début du notebook
from google.colab import files
uploaded = files.upload()

In [ ]:
image_path = "/content/marche_noel_trifouillis_2023.png"

### Easy OCR

In [ ]:
!pip install easyocr --quiet

In [ ]:
import easyocr
import os

# Initialiser EasyOCR
reader = easyocr.Reader(['fr'])  # 'fr' pour le français, ajoutez d'autres langues si nécessaire

# Extraire le texte de chaque image
full_text = ""

# Appliquer l'OCR sur l'image
results = reader.readtext(image_path)

# Formater les résultats
for (bbox, text, prob) in results:
    full_text += text + "\n"
print(full_text)

### GOT OCR

In [ ]:
!pip install tiktoken --quiet
!pip install verovio --quiet

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)
model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True, low_cpu_mem_usage=True, device_map='cuda', use_safetensors=True, pad_token_id=tokenizer.eos_token_id)

# plain texts OCR
res = model.chat(tokenizer, image_path, ocr_type='ocr')

print(res)

## Extraire les textes issus des documents audios

---



In [ ]:
# Bouton pour uploader un fichier au début du notebook
from google.colab import files
uploaded = files.upload()

In [ ]:
# Chemin vers le fichier PDF
audio_path = "/content/voeux2025_trifouillis.wav"

### Whisper

In [ ]:
!pip install -U openai-whisper --quiet

In [ ]:
import whisper

# Transcrire avec Whisper
def transcribe_audio_with_whisper(audio_file, model_name="base"):
    # Charger le modèle Whisper
    model = whisper.load_model(model_name)

    # Transcrire le fichier audio
    result = model.transcribe(audio_file)

    # Afficher la transcription
    print("Transcription :")
    print(result["text"])

# Transcrire l'audio avec Whisper
transcribe_audio_with_whisper(audio_path)

## Génération de l'audio

---






In [ ]:
from transformers import VitsModel, AutoTokenizer
import torch
import numpy as np

model = VitsModel.from_pretrained("facebook/mms-tts-fra")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-fra")

text = """Chères Trifouillaises, chers Trifouillais,
C’est avec une grande émotion et une immense fierté que je me tiens devant vous ce soir pour vous adresser mes vœux les plus sincères en ce début d’année 2025. L’année écoulée a été marquée par de nombreux défis, mais aussi par de belles avancées pour notre commune.
Nous ne pouvons ignorer le contexte régional et national, qui demeure complexe et parfois source d’inquiétudes. Les Pays de la Loire traversent une période de tensions économiques et sociales, et nous sommes pleinement conscients des préoccupations qui peuvent être les vôtres. Mais je veux vous dire ici, avec force et conviction : nous sommes unis, solidaires, et nous avons en nous toutes les ressources pour faire face aux épreuves. Trifouillis-sur-Loire a toujours su se relever et avancer, et 2025 ne fera pas exception.
Je sais que la fermeture de notre gare a suscité de vives inquiétudes, et je comprends votre attachement à ce service de transport essentiel. Mais loin de voir cela comme une régression, nous avons souhaité en faire une opportunité pour notre ville. Dès cette année, Trifouillis-sur-Loire se dotera d’une flotte de bus 100 % électriques, modernes et accessibles, permettant de relier efficacement notre commune aux pôles de transport régionaux. Nous avons travaillé main dans la main avec la Région et les autorités de transport pour garantir une solution fiable et adaptée aux besoins de chacun. Soyez assurés que nous continuerons à œuvrer pour améliorer la mobilité et l’accessibilité de notre territoire.
Au-delà de cette transition, 2025 sera une année placée sous le signe de l’innovation et du développement durable. Nous poursuivrons notre engagement en faveur de l’écologie, avec de nouveaux projets d’aménagement urbain, de préservation de notre patrimoine naturel et de soutien aux initiatives locales.
Je tiens également à saluer l’engagement de nos associations, de nos commerçants, de nos artisans, et de toutes celles et ceux qui participent activement à la vie de notre commune. Trifouillis-sur-Loire, c’est avant tout une communauté soudée, forte de son histoire et tournée vers l’avenir.
En ce début d’année, je veux vous adresser un message d’optimisme et d’espoir. Ensemble, nous continuerons à bâtir une ville plus dynamique, plus solidaire et plus durable.
Je vous souhaite, à toutes et à tous, une très belle année 2025, remplie de santé, de bonheur et de réussite.
Vive Trifouillis-sur-Loire ! Vive notre belle région ! Et vive la République !
"""

inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    output = model(**inputs).waveform

# Conversion en NumPy et normalisation
audio_numpy = output.cpu().numpy().squeeze()  # Convertir en tableau NumPy et retirer la dimension inutile
audio_int16 = (audio_numpy * 32767).astype(np.int16)  # Conversion en int16 pour compatibilité WAV

# Sauvegarde en fichier audio
scipy.io.wavfile.write("voeux2025_trifouillis.wav", rate=model.config.sampling_rate, data=audio_int16)

# Lecture de l'audio
from IPython.display import Audio
Audio(audio_int16, rate=model.config.sampling_rate)